# Décrire les numérisations faites en archives
*MAJ: 4.07.23 par Adrien Jeanrenaud*

Ce notebook permet de gérer les images numérisées en archives (pdfs et images de différents formats) afin de mieux les décrire en modifiant le titre et en remplissant un tableau CSV annexe. De fait, le Notebook nécessite des **fichiers numérisés et un CSV**.

Les **fichiers numérisés** sont enregsitrés dans des dossiers les concernants et les pdfs sont séparés en images uniques. Pour chacun de ces fichiers il faut:

> définir un dossier de sauvegarde

> définir un titre précis

Le **CSV** (un exemple est disponible sur Github) doit contenir au moins une colonne contenant le titre et une autre avec des tags:

| filename | tags | pays | date | etc...|
|----------|----------|----------|----------|----------|
| image_1 | #tag1 #tag2 | Suisse | 26.05.88 |          |


## Marche à suivre

Le notebook est fait en deux parties: 

1. Gérer les PDFs
2. Gérer les images (".jpg", ".jpeg", ".png", ".heic", ".tif", ".tiff", ".gif")

Chacune de ces parties à ensuite la même marche à suivre:

* Importer le ou les fichiers numérisés
* Importer le fichier CSV
* Afficher les images, l'une après l'autre, et remplir les champs
* Enregistrer les informations dans le CSV

In [ ]:
# Installer les librairies si nécessaire

!pip install tkinter pandas fitz tempfile matplotlib pillow pillow_heif

In [ ]:
# importer les librairies

import tkinter as tk
from tkinter import Tk
from tkinter import filedialog
import pandas as pd
import fitz
import tempfile
import matplotlib.pyplot as plt
from PIL import Image
import os
from pillow_heif import register_heif_opener

## 1. PDFs
### 1.1 Cibler un fichier et importer une suite d'images (PDF)

- Choisir un fichier PDF

**Si la fenêtre de sélection ne s'ouvre pas, il faut cliquer sur l'icône Python**

In [ ]:
# cibler un pdf

def import_file():
    filepath = filedialog.askopenfilename(filetypes=[("Fichiers PDF", "*.pdf"), ("Tous les fichiers", "*.*")])
    if filepath:
        return filepath
    else:
        return None

# Création de la fenêtre principale (invisible)
window = tk.Tk()
window.withdraw()

# Appel de la fonction d'importation
selected_file = import_file()

# Affichage du fichier sélectionné
if selected_file:
    print("Fichier PDF sélectionné :", selected_file)
else:
    print("Aucun fichier sélectionné.")

### 1.2 Préparer un document annexe pour décrire l'image (CSV)

- Cibler un csv (en faire un pour toute les images)

**Si la fenêtre de sélection ne s'ouvre pas, il faut cliquer sur l'icône Python**

In [ ]:
# Ouvrir une boîte de dialogue pour sélectionner le fichier CSV
root = Tk()
root.withdraw()
file_path = filedialog.askopenfilename(filetypes=[('CSV Files', '*.csv')])

# Vérifier si un fichier a été sélectionné
if file_path:
    # Lire le fichier CSV en tant que DataFrame
    df = pd.read_csv(file_path)
    
    # Afficher le DataFrame
    print(df.head())
else:
    print("Aucun fichier sélectionné.")


In [ ]:
# définir les zones suivantes: titre, tags

print("Entrez le nom de colonne pour :")
titre = input("Titre :")
tags = input("Tags :")

### 1.3 Afficher l'image: changer le titre et la décrire

In [ ]:
# Spécifiez le chemin d'accès à votre fichier PDF
pdf_path = selected_file

print(selected_file)
# Spécifiez le dossier de destination pour les images
output_folder = input("Entrez le chemin du dossier de destination : ")

# Créez le dossier de destination s'il n'existe pas déjà
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Ouvrez le fichier PDF en mode lecture binaire
doc = fitz.open(pdf_path)

# Obtenez le nombre total de pages dans le PDF
total_pages = doc.page_count

# Chargez le fichier CSV contenant les noms de colonnes
df = df

# Extraire toutes les valeurs de la colonne 'tags'
all_tags = df[str(tags)].str.split(expand=True).values.flatten()

# Obtenir les valeurs uniques
unique_tags = pd.unique(all_tags)

# Obtenez les noms de colonnes à partir du fichier CSV
column_names = df.columns

# Parcourez chaque page du PDF
for page_number in range(total_pages):
    # Récupérez la page individuelle
    page = doc.load_page(page_number)

    # Récupérez la liste des blocs d'image sur la page
    image_blocks = page.get_images(full=True)

    # Parcourez chaque bloc d'image
    for index, image_block in enumerate(image_blocks):
        # Récupérez les informations sur l'image
        xref = image_block[0]
        base_image = doc.extract_image(xref)
        image_data = base_image["image"]

        # Enregistrez l'image extraite dans un fichier temporaire
        with tempfile.NamedTemporaryFile(suffix=".png") as temp_file:
            temp_filename = temp_file.name
            temp_file.write(image_data)

            # Ouvrez le fichier temporaire avec Pillow
            pil_image = Image.open(temp_filename)

            # Affichez l'image
            plt.figure(figsize=(20, 16))
            plt.imshow(pil_image)
            plt.show()
            
            # Demandez à l'utilisateur de renommer l'image
            new_name = input("Entrez un nouveau nom pour l'image : ")
            nname = new_name + ".png"
            fname = os.path.join(output_folder, nname)
            
            # Demandez à l'utilisateur de saisir les valeurs pour chaque champ
            field_values = {}
            for column_name in column_names:
                if column_name == str(titre):
                    field_value = new_name
                elif column_name == str(tags):
                    # Afficher les valeurs uniques
                    print(unique_tags)
                    field_value = input(f"Entrez la valeur pour le champ '{column_name}' : ")
                else:
                    field_value = input(f"Entrez la valeur pour le champ '{column_name}' : ")
                field_values[column_name] = field_value

            # Sauvegardez l'image avec le nouveau nom
            pil_image.save(fname)


### 1.4 Enregistrer (et vérifier) toutes les informations lorsque le document ciblé est terminé

- modifier le csv, enregistrer et voir les dernières lignes

In [ ]:
# Ajouter les infos au CSV

# Créez un nouveau DataFrame à partir des valeurs de field_values
new_row = pd.DataFrame([field_values], columns=df.columns)

# Concaténez le DataFrame existant avec le nouveau DataFrame
df = pd.concat([df, new_row], ignore_index=True)
# Supprimer les doublons de lignes dans le DataFrame
df = df.drop_duplicates()

# Sauvegardez le DataFrame mis à jour dans le fichier CSV
df.to_csv(file_path, index=False)
df.tail()

## 2. Un dossier d'images
### 2.1 Choisir un dossier d'images à traiter

- séléctionner un dossier dans votre machine

**Si la fenêtre de sélection ne s'ouvre pas, il faut cliquer sur l'icône Python**

In [ ]:
# Fonction pour importer plusieurs fichiers
def import_files():
    # Ouvrir une boîte de dialogue pour sélectionner les fichiers
    Tk().withdraw()
    filepaths = filedialog.askopenfilenames()
    
    if filepaths:
        return filepaths
    else:
        return None

# Appel de la fonction d'importation des fichiers
selected_files = import_files()

# Affichage des fichiers sélectionnés
if selected_files:
    print("Fichiers sélectionnés :")
    for file in selected_files:
        print(file)
else:
    print("Aucun fichier sélectionné.")


### 2.2 Préparer un document annexe pour décrire l'image (CSV)

- Cibler un csv (en faire un pour toute les images)

**Si la fenêtre de sélection ne s'ouvre pas, il faut cliquer sur l'icône Python**

In [ ]:
# Ouvrir une boîte de dialogue pour sélectionner le fichier CSV
root = Tk()
root.withdraw()
file_path = filedialog.askopenfilename(filetypes=[('CSV Files', '*.csv')])

# Vérifier si un fichier a été sélectionné
if file_path:
    # Lire le fichier CSV en tant que DataFrame
    df = pd.read_csv(file_path)
    
    # Afficher le DataFrame
    print(df.head())
else:
    print("Aucun fichier sélectionné.")

In [ ]:
# définir les zones suivantes: titre, tags

print("Entrez le nom de colonne pour :")
titre = input("Titre :")
tags = input("Tags :")

### 2.3 Afficher l'image: changer le titre et la décrire


In [ ]:
# Spécifiez le dossier de destination pour les images
output_folder = input("Entrez le chemin du dossier de destination : ")

# Créez le dossier de destination s'il n'existe pas déjà
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Extensions d'image prises en charge
image_extensions = (".jpg", ".jpeg", ".png", ".heic", ".tif", ".tiff", ".gif")

# Chargez le fichier CSV contenant les noms de colonnes
df = df

# Extraire toutes les valeurs de la colonne 'tags'
all_tags = df[str(tags)].str.split(expand=True).values.flatten()

# Obtenir les valeurs uniques
unique_tags = pd.unique(all_tags)

# Obtenez les noms de colonnes à partir du fichier CSV
column_names = df.columns

# Ouvrez chaque image sélectionnée
for image_path in selected_files:
    # Vérifiez si l'extension de l'image est prise en charge
    if image_path.lower().endswith(image_extensions):
        # Vérifiez si l'extension est HEIC
        if image_path.lower().endswith(".heic"):
            # Convertissez l'image HEIC en format JPEG
            register_heif_opener()
            pil_image = Image.open(image_path)
        else:
            # Ouvrez l'image avec Pillow
            pil_image = Image.open(image_path)

        # Affichez l'image
        plt.figure(figsize=(20, 16))
        plt.imshow(pil_image)
        plt.show()

        # Demandez à l'utilisateur de renommer l'image
        new_name = input("Entrez un nouveau nom pour l'image : ")
        nname = new_name + ".png"
        fname = os.path.join(output_folder, nname)

        # Demandez à l'utilisateur de saisir les valeurs pour chaque champ
        field_values = {}
        for column_name in column_names:
            if column_name == str(titre):
                field_value = new_name
            elif column_name == str(tags):
                # Afficher les valeurs uniques
                print(unique_tags)
                field_value = input(f"Entrez la valeur pour le champ '{column_name}' : ")
            else:
                field_value = input(f"Entrez la valeur pour le champ '{column_name}' : ")
            field_values[column_name] = field_value

        # Sauvegardez l'image avec le nouveau nom
        pil_image.save(fname)
    else:
        print(f"L'image '{image_path}' a une extension non prise en charge.")


### 2.4 Enregistrer (et vérifier) toutes les informations lorsque le document ciblé est terminé


In [ ]:
# Ajouter les infos au CSV

# Créez un nouveau DataFrame à partir des valeurs de field_values
new_row = pd.DataFrame([field_values], columns=df.columns)

# Concaténez le DataFrame existant avec le nouveau DataFrame
df = pd.concat([df, new_row], ignore_index=True)
# Supprimer les doublons de lignes dans le DataFrame
df = df.drop_duplicates()

# Sauvegardez le DataFrame mis à jour dans le fichier CSV
df.to_csv(file_path, index=False)
df.tail()